Connecting google drive

In [45]:
#from google.colab import drive
#drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Installing necessary framework

In [1]:
!pip install mwclient

Getting the comments for sentiment analysis

In [22]:
import mwclient
import time

site = mwclient.Site("en.wikipedia.org")
page = site.pages["Bitcoin"]
revs = list(page.revisions())
revs  = sorted(revs, key=lambda rev: rev["timestamp"])

Getting comments only after year 2020

In [23]:
revs = [revs for revs in revs if revs['timestamp'].tm_year >= 2020]
revs[0]

OrderedDict([('revid', 936990582),
             ('parentid', 932691117),
             ('minor', ''),
             ('user', 'Citation bot'),
             ('timestamp',
              time.struct_time(tm_year=2020, tm_mon=1, tm_mday=22, tm_hour=7, tm_min=30, tm_sec=16, tm_wday=2, tm_yday=22, tm_isdst=-1)),
             ('comment',
              'Alter: url, title, pages, doi-broken-date. Add: work, hdl, doi-broken-date, pmid, pages, issue, volume. Removed parameters. Formatted [[WP:ENDASH|dashes]]. Some additions/deletions were actually parameter name changes. | You can [[WP:UCB|use this bot]] yourself. [[WP:DBUG|Report bugs here]]. | Activated by [[User:Ost316]] | [[Category:Pages with DOIs inactive as of 2019 August]] | via #UCB_Category')])

In [25]:
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity

# we will add this later for better prediction

In [3]:
import pandas as pd
comment_df = pd.DataFrame(revs)

changing the timestamp similar to historical_df

In [4]:
comment_df['timestamp'] = comment_df['timestamp'].apply(lambda ts: pd.Timestamp(year=ts.tm_year, month=ts.tm_mon, day=ts.tm_mday, hour=ts.tm_hour, minute=ts.tm_min, second=ts.tm_sec))
comment_df['timestamp'] = pd.to_datetime(comment_df['timestamp'])

In [6]:
comment_df

,revid,parentid,minor,user,timestamp,comment,commenthidden
0,936990582,932691117,,Citation bot,2020-01-22 07:30:16,"Alter: url, title, pages, doi-broken-date. Add...",NaN
1,937104961,936990582,NaN,Ladislav Mecir,2020-01-23 00:02:08,/* Wallets */ use lowercase per talk,NaN
2,937267012,937104961,NaN,Ladislav Mecir,2020-01-23 22:52:55,/* Units */ edit subsection title,NaN
3,937655459,937267012,NaN,Jtbobwaysf,2020-01-26 10:52:31,/* 2017–2019 */ add,NaN
4,937655568,937655459,NaN,Jtbobwaysf,2020-01-26 10:53:47,/* 2017–2019 */ add,NaN
...,...,...,...,...,...,...,...
1692,1225111908,1225111853,,Thibaut120094,2024-05-22 12:43:26,whoops,NaN
1693,1225513035,1225111908,NaN,AgisdeSparte,2024-05-24 22:17:41,Rescuing 1 sources and tagging 0 as dead.) #IA...,NaN
1694,1225814205,1225513035,,Dogman15,2024-05-26 21:38:53,/* Use for payments */ finishing list with the...,NaN
1695,1228816165,1225814205,NaN,Jtbobwaysf,2024-06-13 10:18:39,"/* 2020–present */ add breadth, probably shoul...",NaN


getting historical data from csv file

In [5]:
historical_df = pd.read_csv("historical_data.csv")

Checking columns


In [6]:
print("Columns in comment_df:", comment_df.columns)
print("Columns in historical_df:", historical_df.columns)

Columns in comment_df: Index(['revid', 'parentid', 'minor', 'user', 'timestamp', 'comment',
       'commenthidden'],
      dtype='object')
Columns in historical_df: Index(['timestamp', 'open', 'high', 'low', 'close', 'volume'], dtype='object')


In [7]:
historical_df['timestamp'] = pd.to_datetime(historical_df['timestamp'])

converting all the comments to string, and joining them in hourly interval, and counting the comments

In [8]:
comment_df['comment'] = comment_df['comment'].astype(str)
comment_df.set_index('timestamp', inplace=True)
comment_df = comment_df.resample('H').agg(
    comment_count=('comment', 'size'),
    merged_comments=('comment', lambda comments: ' '.join(comments))
).reset_index()
comment_df['merged_comments'].replace('', pd.NA, inplace=True)

comment_df['merged_comments'] = comment_df['merged_comments'].fillna(pd.NA)
comment_df['comment_count'] = comment_df['comment_count'].fillna(0.0)

C:\Users\rafid\AppData\Local\Temp\ipykernel_29020\1787451462.py:3: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  comment_df = comment_df.resample('H').agg(
C:\Users\rafid\AppData\Local\Temp\ipykernel_29020\1787451462.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  comment_df['merged_comments'].replace('', pd.NA, inplace=True)


In [9]:
comment_df

,timestamp,comment_count,merged_comments
0,2020-01-22 07:00:00,1,"Alter: url, title, pages, doi-broken-date. Add..."
1,2020-01-22 08:00:00,0,<NA>
2,2020-01-22 09:00:00,0,<NA>
3,2020-01-22 10:00:00,0,<NA>
4,2020-01-22 11:00:00,0,<NA>
...,...,...,...
38598,2024-06-17 13:00:00,0,<NA>
38599,2024-06-17 14:00:00,0,<NA>
38600,2024-06-17 15:00:00,0,<NA>
38601,2024-06-17 16:00:00,0,<NA>


In [12]:
comment_df.columns

Index(['timestamp', 'comment_count', 'merged_comments'], dtype='object')

In [13]:
df = pd.merge(historical_df, comment_df, on='timestamp', how='left')

In [ ]:
df

In [18]:
!pip install textblob


  Using cached textblob-0.18.0.post0-py3-none-any.whl.metadata (4.5 kB)
Using cached textblob-0.18.0.post0-py3-none-any.whl (626 kB)


In [14]:
from textblob import TextBlob

def get_sentiment(text):
    blob = TextBlob(str(text))
    return blob.sentiment.polarity

# Applying sentiment analysis to the dataframe
comment_df['sentiment_score'] = comment_df['merged_comments'].apply(get_sentiment)

# we will add this later for better prediction

In [15]:
comment_df

,timestamp,comment_count,merged_comments,sentiment_score
0,2020-01-22 07:00:00,1,"Alter: url, title, pages, doi-broken-date. Add...",0.0
1,2020-01-22 08:00:00,0,<NA>,0.0
2,2020-01-22 09:00:00,0,<NA>,0.0
3,2020-01-22 10:00:00,0,<NA>,0.0
4,2020-01-22 11:00:00,0,<NA>,0.0
...,...,...,...,...
38598,2024-06-17 13:00:00,0,<NA>,0.0
38599,2024-06-17 14:00:00,0,<NA>,0.0
38600,2024-06-17 15:00:00,0,<NA>,0.0
38601,2024-06-17 16:00:00,0,<NA>,0.0


In [ ]:
# Example applying tokenizer.encode to 'merged_comments' column
df['tokenized_comments'] = df['merged_comments'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)
)

# Concatenating tokenized_comments with numerical features
X = df[features + ['tokenized_comments']]

In [20]:
# Shifting 'close' prices to create target variable for price prediction
df['target'] = df['close'].shift(-1)  # Shifting by one hour assuming hourly data

In [24]:
df.to_csv('merged_data.csv', index=False)

: 